In [ ]:
map_file_virgo='GW170817_skymap.fits'
map_file='bayestar_no_virgo.fits.gz'
url = 'https://dcc.ligo.org/public/0146/G1701985/001/bayestar_no_virgo.fits.gz'
#url_virgo ='https://dcc.ligo.org/public/0146/G1701985/001/bayestar.fits.gz'
url_virgo ='https://dcc.ligo.org/public/0157/P1800381/007/GW170817_skymap.fits.gz'


In [ ]:
%matplotlib notebook

#map_healpix = hp.read_map(map_file)
nside=1024
nested=True

In [ ]:
import ligo.skymap.io
import numpy as np
import healpy as hp
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy import units as u
import ligo.skymap.plot
from matplotlib import pyplot as plt
golden_ratio=(1.+np.sqrt(5))/2.
ysize = 3.5 #inches
plt.rcParams['figure.figsize']=(ysize , ysize) #* golden_ratio
from matplotlib import rc
rc('font',**{'family':'sans','serif':['Times New Roman']})
rc('text', usetex=False)

In [ ]:
#Local
(map_array, map_header)=ligo.skymap.io.read_sky_map(map_file)
(map_array_virgo, map_header_virgo)=ligo.skymap.io.read_sky_map(map_file_virgo)

#WEB
#(map_array, map_header)=ligo.skymap.io.read_sky_map(url)
#(map_array_virgo, map_header_virgo)=ligo.skymap.io.read_sky_map(url_virgo)

map_header_virgo

In [ ]:
#Gets continement probability
indices = np.argsort(-map_array)
region = np.empty(map_array.shape)
region[indices] = 100 * np.cumsum(map_array[indices])

indices_v = np.argsort(-map_array_virgo)
region_v = np.empty(map_array_virgo.shape)
region_v[indices_v] = 100 * np.cumsum(map_array_virgo[indices_v])

center = SkyCoord.from_name('NGC 4993')
center_str = '%fd %fd'%(center.ra.deg, center.dec.deg)

In [ ]:
mask=561.
detector=450.
height=630.

fov_side= (90-np.rad2deg(np.arctan( height/( mask/2+detector/2))))

fov_diag = (90-np.rad2deg(np.arctan( height/ np.sqrt(2) /( mask/2+detector/2))))

fov_diag, fov_side

In [ ]:
def wrapped_angle(a):
    """Convert an angle to a reference angle between 0 and 2*pi."""
    return np.mod(a, 2 * np.pi)

def make_rhombus_poly(diag1, diag2, tilt, theta, phi):
    """Create a Polygon patch representing a rectangle with half-angles width
    and height rotated from the north pole to (theta, phi).
    """
    # Convert width and height to radians, then to Cartesian coordinates.
#     cosw = np.cos(np.deg2rad(width))
#     cosh = np.cos(np.deg2rad(height))
#     sinw = np.sin(np.deg2rad(width))
#     sinh = np.sin(np.deg2rad(height))
    
#     # Generate vertices of rectangle.
#     tmp = np.asarray([[cosw*sinh, cosh*sinw], [-cosw*sinh, cosh*sinw], 
#                       [-cosw*sinh, -cosh*sinw], [cosw*sinh, -cosh*sinw]])
    
    
    # Convert width and height to radians, then to Cartesian coordinates.
    sin1 = np.sin(np.deg2rad(diag1))
    sin2 = np.sin(np.deg2rad(diag2))
    
#     # Generate vertices of rectangle.
#     tmp = np.asarray([[cosw*sinh, cosh*sinw], [-cosw*sinh, cosh*sinw], 
#                       [-cosw*sinh, -cosh*sinw], [cosw*sinh, -cosh*sinw]])
    tmp=np.array( [ [sin1,0], [0,sin2], [-sin1,0], [0,-sin2] ])
    
    

    # Subdivide.
    #tmp2 = subdivide_vertices(tmp, subdivisions)
    
    #print(tmp)
    # Project onto sphere by calculating z-coord from normalization condition.
    v = np.hstack((tmp, np.sqrt(1. - np.expand_dims(np.square(tmp).sum(1), 1))))
    
    
    rot_angle = np.arctan(sin2/sin1)
    
    #v = np.hstack((tmp2, np.sqrt(1. - np.expand_dims(np.square(tmp2).sum(1), 1))))
   
    # Transform vertices.
    #rotates out of diagonal
    v_rot = np.dot(v, hp.rotator.euler_matrix_new(rot_angle,0, 0, Y=True))
    
    #makes a tilt
    v_rot = np.dot(v_rot, hp.rotator.euler_matrix_new(0, np.deg2rad(tilt), 0, Y=True))
    
    #centers on desired projection center
    v_rot = np.dot(v_rot, hp.rotator.euler_matrix_new(phi, theta, 0, Y=True))

    #print(v_rot)
    return v_rot
    
    # Convert to spherical polar coordinates.
    #thetas, phis = hp.vec2ang(v)

    # Return list of vertices as longitude, latitude pairs.
    #return np.column_stack((wrapped_angle(phis), 0.5 * np.pi - thetas))

In [ ]:
def make_rect_poly(width=61.0, height=31.0, theta=0, phi=0):
    """Create a Polygon patch representing a rectangle with half-angles width
    and height rotated from the north pole to (theta, phi).
    """
    # Convert width and height to radians, then to Cartesian coordinates.
    sinw = np.sin(np.deg2rad(width/2))
    sinh = np.sin(np.deg2rad(height/2))
    
    x = np.array([sinw, -sinw, -sinw, sinw])
    y = np.array([sinh, sinh, -sinh, -sinh])
    # Project onto sphere by calculating z-coord from normalization condition.
    z = np.sqrt( 1 - x**2 - y**2 )
    v = np.stack((x, y, z), axis=-1)    
    
#     for v1, v2 in zip (v[0:-1],v[1:]):
#         d = np.dot(v1,v2)
#         angle = np.rad2deg(np.arccos(d))
#         print(d, angle)
        
    #centers on desired projection center
    v_rot = np.dot(v, hp.rotator.euler_matrix_new(phi, theta, 0, Y=True))

    return v_rot

#make_rect_poly_test(61,31,0,0)

In [ ]:
def getfov_img(width=48.0, height=48.0, tilt=20, ra=center.ra.deg, dec=center.dec.deg ):
    
    pix_ver_vec = make_rhombus_poly(width, height, tilt, np.deg2rad(90.-dec), np.deg2rad(ra))
    #pix_ver = hp.ang2pix(nside, theta_vert, phi_vert)
    #pix_ver_vec=hp.ang2vec(theta_vert, phi_vert)
    fov=hp.query_polygon(nside, pix_ver_vec, inclusive=True)
    return fov

def getfov_img_rect(width, height, ra=center.ra.deg, dec=center.dec.deg ):
    
    pix_ver_vec = make_rect_poly(width, height, np.deg2rad(90.-dec), np.deg2rad(ra))
    #pix_ver = hp.ang2pix(nside, theta_vert, phi_vert)
    #pix_ver_vec=hp.ang2vec(theta_vert, phi_vert)
    fov=hp.query_polygon(nside, pix_ver_vec, inclusive=True)
    return fov


In [ ]:


fig = plt.figure(dpi=100)
ax = plt.axes(
    [0.05, 0.05, 0.9, 0.9],
    projection='astro globe', #center='0d 90d')
    center=center_str)
    
    
# ax_inset = plt.axes(
#     [0.59, 0.3, 0.4, 0.4],
#     projection='astro zoom',
#     center=center_str,
#     radius=10*u.deg)
# for key in ['ra', 'dec']:
#     ax_inset.coords[key].set_ticklabel_visible(False)
#     ax_inset.coords[key].set_ticks_visible(False)
ax.grid()
#ax.mark_inset_axes(ax_inset)
#ax.connect_inset_axes(ax_inset, 'upper left')
#ax.connect_inset_axes(ax_inset, 'lower left')
#ax_inset.scalebar((0.1, 0.1), 5 * u.deg).label()
#ax_inset.compass(0.9, 0.1, 0.2)
#ax.imshow_hpx(map_file, cmap='Blues')
#ax.contourf_hpx(map_file, cmap='Blues')

#ax.contour_hpx(map_array, cmap='cylon', levels=levels)
levels=[50, 90]
ax.contourf_hpx(region, cmap='cylon', levels=levels)

ax.contour_hpx(region_v, cmap='Greens', levels=levels)

fov_region = np.zeros(map_array.shape)
fov = getfov_img(fov_diag, fov_diag, 20, ra=center.ra.deg, dec=center.dec.deg )
fov_region[fov]=1.

fov = getfov_img(fov_diag, fov_diag, -20, ra=center.ra.deg, dec=center.dec.deg )
fov_region[fov]=1.
ax.imshow_hpx(fov_region, cmap='Blues', alpha=0.24)

#ax_inset.imshow_hpx(url, cmap='cylon')
# ax_inset.plot(
#     center.ra.deg, center.dec.deg,
#     transform=ax_inset.get_transform('world'),
#     marker=ligo.skymap.plot.reticle(),
#     markersize=30,
#     markeredgewidth=3)

In [ ]:
plt.savefig('GW170817_THESEUS_FOV_onlyXGIS.tiff', dpi=600, bbox_inches = "tight")
plt.savefig('GW170817_THESEUS_FOV_onlyXGIS.png', dpi=100,bbox_inches = "tight")

In [ ]:
fov_region = np.zeros(map_array.shape)
fov = getfov_img_rect(60,30, ra=center.ra.deg, dec=center.dec.deg )
fov_region[fov]=1.

ax.imshow_hpx(fov_region, cmap='Greens', alpha=0.2)

# fov_region = np.zeros(map_array.shape)
# tmp=make_rect_poly_test(61,31,0,0)
# fov = hp.vec2pix(nside, tmp[:,0],tmp[:,1], tmp[:,2])

# fov_region[fov] = 1
# ax.imshow_hpx(fov_region, cmap='Reds', alpha=0.5)


In [ ]:
plt.savefig('GW170817_THESEUS_FOV_withSXI.tiff', dpi=600, bbox_inches = "tight")
plt.savefig('GW170817_THESEUS_FOV_withSXI.png', dpi=100,bbox_inches = "tight")